In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
df = pd.read_csv("data/player_logs_2023-24_with_bio.csv")
df.head()

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PLAYER_NAME,SEASON,PLAYER_NAME_BIO,POSITION,HEIGHT,HEIGHT_IN,WEIGHT,WEIGHT_LB,BIRTHDATE,TEAM
0,22023,1630173,22301190,"Apr 14, 2024",NYK vs. CHI,W,19,2,3,0.667,...,Precious Achiuwa,2023-24,Precious Achiuwa,Forward,6-8,80,243.0,243.0,1999-09-19T00:00:00,Knicks
1,22023,1630173,22301175,"Apr 12, 2024",NYK vs. BKN,W,8,2,2,1.000,...,Precious Achiuwa,2023-24,Precious Achiuwa,Forward,6-8,80,243.0,243.0,1999-09-19T00:00:00,Knicks
2,22023,1630173,22301167,"Apr 11, 2024",NYK @ BOS,W,16,1,6,0.167,...,Precious Achiuwa,2023-24,Precious Achiuwa,Forward,6-8,80,243.0,243.0,1999-09-19T00:00:00,Knicks
3,22023,1630173,22301139,"Apr 07, 2024",NYK @ MIL,W,5,0,1,0.000,...,Precious Achiuwa,2023-24,Precious Achiuwa,Forward,6-8,80,243.0,243.0,1999-09-19T00:00:00,Knicks
4,22023,1630173,22301119,"Apr 05, 2024",NYK @ CHI,L,19,0,2,0.000,...,Precious Achiuwa,2023-24,Precious Achiuwa,Forward,6-8,80,243.0,243.0,1999-09-19T00:00:00,Knicks


In [3]:
# Calculate per 36 minutes stats
# List of columns to calculate per 36 minutes stats for
per_game_stats = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for stat in per_game_stats:
    # Create a new column name for the per 36 minutes stat
    per_36_stat_name = f'{stat}_PER_36_MIN'
    # Calculate the per 36 minutes stat, handling division by zero by filling with 0
    df[per_36_stat_name] = (df[stat] / df['MIN'] * 36).fillna(0)

display(df.head())

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,FTA_PER_36_MIN,OREB_PER_36_MIN,DREB_PER_36_MIN,REB_PER_36_MIN,AST_PER_36_MIN,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN
0,22023,1630173,22301190,"Apr 14, 2024",NYK vs. CHI,W,19,2,3,0.667,...,0.0,1.894737,7.578947,9.473684,3.789474,0.000000,1.894737,3.789474,5.684211,7.578947
1,22023,1630173,22301175,"Apr 12, 2024",NYK vs. BKN,W,8,2,2,1.000,...,0.0,0.000000,13.500000,13.500000,0.000000,0.000000,0.000000,0.000000,0.000000,22.500000
2,22023,1630173,22301167,"Apr 11, 2024",NYK @ BOS,W,16,1,6,0.167,...,0.0,4.500000,6.750000,11.250000,0.000000,0.000000,2.250000,2.250000,0.000000,4.500000
3,22023,1630173,22301139,"Apr 07, 2024",NYK @ MIL,W,5,0,1,0.000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.200000,0.000000
4,22023,1630173,22301119,"Apr 05, 2024",NYK @ CHI,L,19,0,2,0.000,...,0.0,1.894737,5.684211,7.578947,1.894737,1.894737,0.000000,1.894737,7.578947,0.000000


In [4]:
# Group by player and aggregate the numerical stats by calculating the mean
# Also include the 'POSITION' and take the first value
aggregation_functions = {col: 'mean' for col in df.select_dtypes(include=['number']).columns if col not in ['SEASON_ID', 'GAME_ID', 'VIDEO_AVAILABLE', 'PLAYER_ID']}
aggregation_functions['POSITION'] = 'first'


df_aggregated = df.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(aggregation_functions).reset_index()

display(df_aggregated.head())
display(df_aggregated.info())

,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,OREB_PER_36_MIN,DREB_PER_36_MIN,REB_PER_36_MIN,AST_PER_36_MIN,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN,POSITION
0,2544,LeBron James,35.323944,9.647887,17.873239,0.542239,2.098592,5.112676,0.404592,4.267606,...,0.862764,6.587453,7.450217,8.552123,1.240106,0.523435,3.508694,1.091847,26.195756,Forward
1,101108,Chris Paul,26.431034,3.551724,8.051724,0.425069,1.344828,3.620690,0.331966,0.741379,...,0.414074,4.881236,5.295310,9.144667,1.778351,0.118572,1.792716,2.603366,12.688309,Guard
2,200768,Kyle Lowry,28.183333,2.716667,6.283333,0.409717,1.633333,4.166667,0.380950,1.050000,...,0.719022,3.370198,4.089220,5.339844,1.256857,0.454109,1.821471,3.144748,10.002223,Guard
3,200782,P.J. Tucker,15.709677,0.580645,1.612903,0.197839,0.419355,1.129032,0.241968,0.096774,...,1.672023,3.763643,5.435666,1.170992,1.267622,0.361255,0.553392,4.497375,3.275039,Forward
4,201142,Kevin Durant,37.200000,10.013333,19.146667,0.530987,2.240000,5.426667,0.422333,4.826667,...,0.516608,5.851974,6.368582,4.869153,0.908258,1.173239,3.134328,1.697489,26.385060,Forward


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PLAYER_ID        448 non-null    int64  
 1   PLAYER_NAME      448 non-null    object 
 2   MIN              448 non-null    float64
 3   FGM              448 non-null    float64
 4   FGA              448 non-null    float64
 5   FG_PCT           448 non-null    float64
 6   FG3M             448 non-null    float64
 7   FG3A             448 non-null    float64
 8   FG3_PCT          448 non-null    float64
 9   FTM              448 non-null    float64
 10  FTA              448 non-null    float64
 11  FT_PCT           448 non-null    float64
 12  OREB             448 non-null    float64
 13  DREB             448 non-null    float64
 14  REB              448 non-null    float64
 15  AST              448 non-null    float64
 16  STL              448 non-null    float64
 17  BLK             

None

In [5]:
# Calculate Assist-to-Turnover Ratio, handling division by zero
df_aggregated['AST_TOV_RATIO'] = np.where(
    df_aggregated['TOV'] == 0,
    0,  # Or a small number, or NaN, depending on desired handling
    df_aggregated['AST'] / df_aggregated['TOV']
)

# Calculate True Shooting Percentage, handling division by zero
df_aggregated['TS_PCT'] = np.where(
    (2 * (df_aggregated['FGA'] + 0.44 * df_aggregated['FTA'])) == 0,
    0,  # Or NaN
    df_aggregated['PTS'] / (2 * (df_aggregated['FGA'] + 0.44 * df_aggregated['FTA']))
)

# Calculate Offensive Rebounding Ratio
df_aggregated['OREB_RATIO'] = np.where(
    df_aggregated['REB'] == 0,
    0, # Or NaN
    df_aggregated['OREB'] / df_aggregated['REB']
)

# Calculate Defensive Rebounding Ratio
df_aggregated['DREB_RATIO'] = np.where(
    df_aggregated['REB'] == 0,
    0, # Or NaN
    df_aggregated['DREB'] / df_aggregated['REB']
)

display(df_aggregated.head())

,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN,POSITION,AST_TOV_RATIO,TS_PCT,OREB_RATIO,DREB_RATIO
0,2544,LeBron James,35.323944,9.647887,17.873239,0.542239,2.098592,5.112676,0.404592,4.267606,...,1.240106,0.523435,3.508694,1.091847,26.195756,Forward,2.404082,0.629683,0.117761,0.882239
1,101108,Chris Paul,26.431034,3.551724,8.051724,0.425069,1.344828,3.620690,0.331966,0.741379,...,1.778351,0.118572,1.792716,2.603366,12.688309,Guard,5.171053,0.544011,0.080000,0.920000
2,200768,Kyle Lowry,28.183333,2.716667,6.283333,0.409717,1.633333,4.166667,0.380950,1.050000,...,1.256857,0.454109,1.821471,3.144748,10.002223,Guard,2.976471,0.593902,0.175258,0.824742
3,200782,P.J. Tucker,15.709677,0.580645,1.612903,0.197839,0.419355,1.129032,0.241968,0.096774,...,1.267622,0.361255,0.553392,4.497375,3.275039,Forward,2.000000,0.506625,0.329412,0.670588
4,201142,Kevin Durant,37.200000,10.013333,19.146667,0.530987,2.240000,5.426667,0.422333,4.826667,...,0.908258,1.173239,3.134328,1.697489,26.385060,Forward,1.549180,0.626341,0.082828,0.917172


In [6]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PLAYER_ID        448 non-null    int64  
 1   PLAYER_NAME      448 non-null    object 
 2   MIN              448 non-null    float64
 3   FGM              448 non-null    float64
 4   FGA              448 non-null    float64
 5   FG_PCT           448 non-null    float64
 6   FG3M             448 non-null    float64
 7   FG3A             448 non-null    float64
 8   FG3_PCT          448 non-null    float64
 9   FTM              448 non-null    float64
 10  FTA              448 non-null    float64
 11  FT_PCT           448 non-null    float64
 12  OREB             448 non-null    float64
 13  DREB             448 non-null    float64
 14  REB              448 non-null    float64
 15  AST              448 non-null    float64
 16  STL              448 non-null    float64
 17  BLK             

In [7]:
X = df_aggregated.select_dtypes(include=['number']).drop('PLAYER_ID', axis=1)

# Clean the 'POSITION' column to keep only the first position listed
# Assuming positions are separated by '/'
df_aggregated['POSITION'] = df_aggregated['POSITION'].str.split('/').str[0].str.strip()
y = df_aggregated['POSITION']

display(X.head())
display(y.head())

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,AST_PER_36_MIN,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN,AST_TOV_RATIO,TS_PCT,OREB_RATIO,DREB_RATIO
0,35.323944,9.647887,17.873239,0.542239,2.098592,5.112676,0.404592,4.267606,5.690141,0.730859,...,8.552123,1.240106,0.523435,3.508694,1.091847,26.195756,2.404082,0.629683,0.117761,0.882239
1,26.431034,3.551724,8.051724,0.425069,1.344828,3.620690,0.331966,0.741379,0.896552,0.259034,...,9.144667,1.778351,0.118572,1.792716,2.603366,12.688309,5.171053,0.544011,0.080000,0.920000
2,28.183333,2.716667,6.283333,0.409717,1.633333,4.166667,0.380950,1.050000,1.250000,0.381950,...,5.339844,1.256857,0.454109,1.821471,3.144748,10.002223,2.976471,0.593902,0.175258,0.824742
3,15.709677,0.580645,1.612903,0.197839,0.419355,1.129032,0.241968,0.096774,0.096774,0.064516,...,1.170992,1.267622,0.361255,0.553392,4.497375,3.275039,2.000000,0.506625,0.329412,0.670588
4,37.200000,10.013333,19.146667,0.530987,2.240000,5.426667,0.422333,4.826667,5.640000,0.802547,...,4.869153,0.908258,1.173239,3.134328,1.697489,26.385060,1.549180,0.626341,0.082828,0.917172


0    Forward
1      Guard
2      Guard
3    Forward
4    Forward
Name: POSITION, dtype: object

In [8]:
# Clean the 'POSITION' column to keep only the first position listed
# Assuming positions are separated by '-'
df_aggregated['POSITION'] = df_aggregated['POSITION'].str.split('-').str[0].str.strip()

# Select all numerical columns from the aggregated DataFrame, excluding the player ID
X = df_aggregated.select_dtypes(include=['number']).drop('PLAYER_ID', axis=1)
y = df_aggregated['POSITION']

display(X.head())
display(y.head())

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,AST_PER_36_MIN,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN,AST_TOV_RATIO,TS_PCT,OREB_RATIO,DREB_RATIO
0,35.323944,9.647887,17.873239,0.542239,2.098592,5.112676,0.404592,4.267606,5.690141,0.730859,...,8.552123,1.240106,0.523435,3.508694,1.091847,26.195756,2.404082,0.629683,0.117761,0.882239
1,26.431034,3.551724,8.051724,0.425069,1.344828,3.620690,0.331966,0.741379,0.896552,0.259034,...,9.144667,1.778351,0.118572,1.792716,2.603366,12.688309,5.171053,0.544011,0.080000,0.920000
2,28.183333,2.716667,6.283333,0.409717,1.633333,4.166667,0.380950,1.050000,1.250000,0.381950,...,5.339844,1.256857,0.454109,1.821471,3.144748,10.002223,2.976471,0.593902,0.175258,0.824742
3,15.709677,0.580645,1.612903,0.197839,0.419355,1.129032,0.241968,0.096774,0.096774,0.064516,...,1.170992,1.267622,0.361255,0.553392,4.497375,3.275039,2.000000,0.506625,0.329412,0.670588
4,37.200000,10.013333,19.146667,0.530987,2.240000,5.426667,0.422333,4.826667,5.640000,0.802547,...,4.869153,0.908258,1.173239,3.134328,1.697489,26.385060,1.549180,0.626341,0.082828,0.917172


0    Forward
1      Guard
2      Guard
3    Forward
4    Forward
Name: POSITION, dtype: object

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (358, 42)
Shape of X_test: (90, 42)
Shape of y_train: (358,)
Shape of y_test: (90,)


In [11]:
from sklearn.impute import SimpleImputer


# Replace infinite values with NaN in both training and testing sets
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Initialize the imputer (e.g., using the mean strategy)
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the training data and transform both training and testing data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert the imputed arrays back to DataFrames with original column names
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

print("Shape of X_train_imputed:", X_train_imputed.shape)
print("Shape of X_test_imputed:", X_test_imputed.shape)
display(X_train_imputed.head())

Shape of X_train_imputed: (358, 42)
Shape of X_test_imputed: (90, 42)


,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,AST_PER_36_MIN,STL_PER_36_MIN,BLK_PER_36_MIN,TOV_PER_36_MIN,PF_PER_36_MIN,PTS_PER_36_MIN,AST_TOV_RATIO,TS_PCT,OREB_RATIO,DREB_RATIO
0,5.111111,0.333333,1.111111,0.185111,0.000000,0.222222,0.000000,0.222222,0.222222,0.111111,...,5.109524,3.500000,0.571429,3.871429,5.857143,5.076190,1.600000,0.367647,0.250000,0.750000
1,33.589744,11.538462,21.820513,0.525487,1.384615,3.564103,0.381000,10.230769,11.589744,0.885103,...,5.995712,1.283060,1.811856,4.201155,3.073591,37.366708,1.460000,0.644359,0.220930,0.779070
2,37.485714,11.485714,23.600000,0.484114,4.057143,10.628571,0.374971,6.828571,8.685714,0.794214,...,9.359757,1.349075,0.503179,3.901239,2.021542,32.644455,2.432624,0.617342,0.091190,0.908810
3,9.931034,1.189655,2.568966,0.387828,0.103448,0.396552,0.091948,0.724138,1.068966,0.271276,...,1.957716,0.977859,1.844614,2.004933,6.396430,10.282092,1.185185,0.527570,0.484127,0.515873
4,34.368421,7.710526,15.934211,0.479671,1.210526,4.118421,0.298684,1.394737,1.697368,0.485342,...,3.530777,0.689293,0.829067,1.682585,2.603788,18.844465,2.116667,0.540323,0.262172,0.737828


In [12]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the Gradient Boosting Classifier model
gb_model_cleaned_pos = GradientBoostingClassifier(random_state=42)

gb_model_cleaned_pos.fit(X_train_imputed, y_train)

,loss,'log_loss'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [13]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test set using the Gradient Boosting model after imputation and position cleaning
gb_y_pred_cleaned_pos = gb_model_cleaned_pos.predict(X_test_imputed)

# Calculate accuracy
gb_accuracy_cleaned_pos = accuracy_score(y_test, gb_y_pred_cleaned_pos)
print(f"Gradient Boosting Model Accuracy (after imputation and position cleaning): {gb_accuracy_cleaned_pos:.4f}")

# Generate classification report
gb_report_cleaned_pos = classification_report(y_test, gb_y_pred_cleaned_pos)
print("\nGradient Boosting Model Classification Report (after imputation and position cleaning):")
print(gb_report_cleaned_pos)

Gradient Boosting Model Accuracy (after imputation and position cleaning): 0.8444

Gradient Boosting Model Classification Report (after imputation and position cleaning):
              precision    recall  f1-score   support

      Center       0.75      0.60      0.67        10
     Forward       0.75      0.84      0.79        32
       Guard       0.93      0.90      0.91        48

    accuracy                           0.84        90
   macro avg       0.81      0.78      0.79        90
weighted avg       0.85      0.84      0.84        90



In [14]:
import joblib

# Define the filename for the saved model
model_filename = 'gradient_boosting_model.pkl'

# Save the trained Gradient Boosting model to the file
joblib.dump(gb_model_cleaned_pos, model_filename)

print(f"Trained Gradient Boosting model saved to {model_filename}")

Trained Gradient Boosting model saved to gradient_boosting_model.pkl
